In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = True
%config SqlMagic.displaycon = False

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

# Dataset Preparation
This notebook prepares the test dataset for Timeseries model training. It performs basic quality checks, splits the data into train/test parts and uploads to S3 storage.

## Download and load sample data

In [3]:
!uv run kaggle competitions download -c demand-forecasting-kernels-only -p ../data/raw
!unzip -o -q -d ../data/raw ../data/raw/demand-forecasting-kernels-only.zip

demand-forecasting-kernels-only.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
%%sql
CREATE OR REPLACE VIEW raw_data AS
SELECT * FROM read_csv_auto('../data/raw/train.csv') ORDER BY date;

,Count


In [4]:
%%sql
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'raw_data'
  AND table_schema IN ('main','temp')
ORDER BY ordinal_position;

,column_name,data_type,is_nullable
0,date,DATE,YES
1,store,BIGINT,YES
2,item,BIGINT,YES
3,sales,BIGINT,YES


In [5]:
%sql SELECT * FROM raw_data LIMIT 10;

,date,store,item,sales
0,2013-01-01,4,47,23
1,2013-01-01,7,47,4
2,2013-01-01,6,47,5
3,2013-01-01,3,47,10
4,2013-01-01,9,47,6
5,2013-01-01,5,47,9
6,2013-01-01,8,47,19
7,2013-01-01,10,47,11
8,2013-01-01,1,48,21
9,2013-01-01,6,24,33


## Quality checks

### No gaps in data
Continuous daily cadence (no missing days) for each `(store, item)` combination.

In [6]:
%%sql
WITH ordered AS (
    SELECT
        item,
        store,
        date,
        LAG(date) OVER (PARTITION BY item, store ORDER BY date) AS prev_date
    FROM raw_data
),
gaps AS (
    SELECT
        item,
        store,
        prev_date,
        date AS current_date,
        DATE_DIFF('day', prev_date, date) AS day_diff
    FROM ordered
    WHERE prev_date IS NOT NULL
      AND DATE_DIFF('day', prev_date, date) > 1
)
SELECT *
FROM gaps
ORDER BY item, store, prev_date;

,item,store,prev_date,current_date,day_diff


### No NULLs

In [7]:
%%sql
SELECT * FROM raw_data
WHERE 
    date = NULL
    OR store = NULL
    OR item = NULL
    OR sales = NULL

,date,store,item,sales


## Train/test split
A separate test dataset would be extracted that will be used later to asses the performance of the ML model performance. Each `(store, item)` would get `n_rows` for evaluation. Think of it as last 7 days are used for testing.

In [8]:
test_last_n_rows = 7

In [9]:
%%sql
CREATE OR REPLACE VIEW raw_with_split AS
SELECT
  r.*,
  CASE
    WHEN ROW_NUMBER() OVER (
           PARTITION BY item, store
           ORDER BY date DESC
         ) <= {{test_last_n_rows}}
      THEN 'test'
    ELSE 'train'
  END AS split
FROM raw_data r

,Count


In [10]:
%%sql
SELECT split, count(1) FROM raw_with_split GROUP BY 1;

,split,count(1)
0,test,3500
1,train,909500


Let's create three files

1. Train data set `train.csv`
2. Test data set `test.csv` (last 7 days for each (store, item)). No `sales` column
3. Test data set `test_true.csv` same as above but with `sales` column for evaluation

In [11]:
%%sql
COPY (SELECT * EXCLUDE (split) FROM raw_with_split WHERE split = 'train')
TO '../data/train.csv' (HEADER, DELIMITER ',');

,Count
0,909500


In [15]:
%%sql
COPY (SELECT * EXCLUDE (split) FROM raw_with_split WHERE split = 'test')
TO '../data/test.csv' (HEADER, DELIMITER ',');

,Count
0,3500


## Upload data to S3

In [16]:
import os
from pathlib import Path
import boto3

intput_bucket_name = os.environ["SM_INPUT_BUCKET"]

files_to_upload = [Path("../data/train.csv"), Path("../data/test.csv")]

s3 = boto3.client("s3")

for f in files_to_upload:
    s3.upload_file(f, intput_bucket_name, f"data/{f.name}")
    print(f"Uploaded {f.name}")

Uploaded train.csv
Uploaded test.csv
